# Dimostrazione automatica al primo ordine

In [1]:
#load "../../localInit.fsx"

open Calculemus.Lib.Function
open Calculemus.Lib.Set

open Calculemus.Formulas
open Calculemus.Fol
open Calculemus.Skolem
open Calculemus.Herbrand
open Calculemus.Lib
open Calculemus.Prop
open Calculemus.Pelletier

Nell'Handbook viene mostrato come la combinazione del teorema di Herbrand (ivi 3.23):

> Una formula priva di quantificatori $p$ è soddisfacibile al primo ordine 
> sse l’insieme di **tutte** le sue istanze ground è soddisfacibile (dal 
> punto di vista proposizionale).

più il teorema di compattezza (ivi 2.13) per la logica proposizionale:

> Per ogni insieme $\Gamma$ di formule proposizionali, se **ciascun** 
> sotto insieme **finito** $\Delta \subseteq \Gamma$ è soddisfacibile, 
> allora lo stesso $\Gamma$ è soddisfacibile.

porta al seguente risultato lì enunciato come Corollario 3.26:

> Una formula priva di quantificatori $p$ è insoddisfacibile al primo ordine
> sse **qualche** insieme **finito** di istanze ground è (proposizionalmente) 
> insoddisfacibile.

Questo risultato garantisce che una procedura in grado di generare per una formula $p$ insiemi sempre più grandi di istanze ground, e a condizione che ogni istanza ground della formula prima o poi compaia nell'enumerazione e che la formula stessa sia insoddisfacibile, prima o poi raggiungerà un insieme finito insoddisfacibile di formule proposizionali.

Questa impostazione da origine a una serie di procedure di semi-decisione circa l'insoddisfacibilità di una formula al primo ordine, perché, anche se tale formula contenesse dei quantificatori, sarebbe possibile fornirne una equi-soddisfacibile per mezzo della Skolemizzazione. D'altra parte, si parla di procedure di **semi-decisione** perché per una formula $p$ che invece fosse soddisfacibile questo processo potrebbe non terminare mai.

Si noti che avere procedure di semi-decisione circa l'insoddisfacibilità di una formula equivale ad averne anche per la la validità, perché una formula è valida se la sua negazione è insoddisfacibile.

## Procedura di Gilmore

La procedura di Gilmore, partendo da una formula priva di quantificatori (espressa in dnf), genera insiemi sempre più grandi di istanze ground sotto forma di $m$-uple di termini ground dove $m$ è il numero di variabili (ovviamente libere) nella formula. Per garantire che qualsiasi istanza ground della formula prima o poi venga generata dalla procedura, un approccio utilizzato è quello di generare prima tutte le $m$-uple che non coinvolgono simboli di funzione (cioè solo combinazioni di termini costanti), quindi tutte quelle che coinvolgono un'applicazione di funzione, ($f(c);g(c)$) poi due ($f(f(c));f(g(c));\ldots$), tre, ecc.

Supponiamo di voler verificare la validità della formula $\exists x.\ \forall y.\ P(x) \Rightarrow P(y)$ con questa procedura. 

1. Dal momento che si tratta di una procedura per la verifica dell'insoddisfacibilità si parte dalla negazione della formula (per altro chiusa universalmente nel caso contenesse variabili libere) e inoltre, dal momento che occorre una formula priva di quantificatori, la negazione viene posta in forma normale di skolem con anche i quantificatori universali rimossi:

In [2]:
let fm = !!"exists x. forall y. P(x) ==> P(y)"
let sfm = skolemize (Not (generalize fm))
sfm

<<P(x) /\ ~P(f_y(x))>>

2. se ne estraggono le variabili più le costanti e le funzioni per la sua base di Herbrand:

In [3]:
let fvs = fv sfm
let consts, funcs = herbfuns sfm

printfn "%s" (fvs.ToString())
printfn "%s" (consts.ToString())
printfn "%s" (funcs.ToString())

[x]
[(c, 0)]
[(f_y, 1)]


3. le istanze ground saranno generate dalla sua DNF in forma di insiemi di letterali:

In [4]:
let dnf = simpdnf sfm
dnf

[[<<P(x)>>;<<~P(f_y(x))>>]]

4. si iniziano quindi a generare le $m$-uple (in questo caso $m=1$ dal momento che c'è una sola variabile) di istanze ground di sole costanti:

In [5]:
let cntms = image (fun (c, _) -> Fn (c, [])) consts
let newtups = groundtuples cntms funcs 0 (List.length fvs)

let fl' = (distrib (image (image (subst (fpf fvs (newtups |> List.head)))) dnf) [[]])

fl'

[[<<P(c)>>;<<~P(f_y(c))>>]]

5. si testa se sono insoddisfacibili:

In [6]:
fl'
|> List.filter (non trivial)
|> (<>) []
|> not

False

6. dal momento che non lo sono, si generano le $m$-uple successive:

In [7]:
let newtups' = groundtuples cntms funcs 1 (List.length fvs)
(distrib (image (image (subst (fpf fvs (newtups' |> List.head)))) dnf) [[]])

[[<<P(f_y(c))>>;<<~P(f_y(f_y(c)))>>]]

(nella procedura le nuove istanze generate sono aggiunte immediatamente alle precedenti)

In [8]:
let fl'' = (distrib (image (image (subst (fpf fvs (newtups' |> List.head)))) dnf) fl')
fl'' 

[[<<P(c)>>;<<P(f_y(c))>>;<<~P(f_y(c))>>;<<~P(f_y(f_y(c)))>>]]

7. si testa quindi per l'insoddisfacibilità dell'insieme di istanze ground generate:

In [9]:
fl''
|> List.filter (non trivial)
|> (<>) []
|> not

True

Dal momento che le due $m$-uple generate contengono due istanze complementari (`<<P(f_y(c))>>` e `<<~P(f_y(c))>>`), la procedura termina, indicando che sono state usate due istanze e che la formula è valida come sostenuto:

In [10]:
gilmore fm

0 ground instances tried; 1 items in list.
0 ground instances tried; 1 items in list.
1 ground instances tried; 1 items in list.
1 ground instances tried; 1 items in list.


2